In [1]:
import numpy as np
import pandas as pd
from operator import attrgetter
from sklearn.linear_model import LogisticRegression
import random

In [2]:
Data = pd.read_excel('./Midterm_Data/Data.xlsx') # training data
Test1 = pd.read_excel('./Midterm_Data/Test1.xlsx') # validation data
Test2 = pd.read_excel('./Midterm_Data/Test2.xlsx') # testing data

Data.fillna(Data.mean(), inplace=True)
# Data # training data

# data preprocessing

- 1 way ratio: 0 ~ 94 (xi)
- 2 way ratio: 95 ~ 4559 (xi * xj)

In [3]:
def data_preprocessing(data):
    
    # Data_bio_ratio
    x = pd.DataFrame(index=range(data.shape[0]),columns=range(95))
    for i in range(19):
        for j in range(5):
            if i < 9:
                x[i * 5 + j] = data[f'F0{i+1}'] / data[f'R0{j+1}']
            else:
                x[i * 5 + j] = data[f'F{i+1}'] / data[f'R0{j+1}']

    one_way = x.to_numpy()
#     two_way = np.zeros((data.shape[0], 95 * 94 // 2))
#     count = 0
#     for i in range(one_way.shape[1]):
#         for j in range(i+1, one_way.shape[1]):
#             two_way[:, count] = one_way[:, i] * one_way[:, j]
#             count += 1
    
#     return np.hstack((one_way, two_way)), data[['C01','C02','C03','C04','C05']].to_numpy().T
    return one_way, data[['C01','C02','C03','C04','C05']].to_numpy().T

In [4]:
x_train, y_train = data_preprocessing(Data)
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

x_val, y_val = data_preprocessing(Test1)
print("x_val.shape: ", x_val.shape)
print("y_val.shape: ", y_val.shape)

x_test, y_test = data_preprocessing(Test2)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

x_train.shape:  (4240, 95)
y_train.shape:  (5, 4240)
x_val.shape:  (475, 95)
y_val.shape:  (5, 475)
x_test.shape:  (987, 95)
y_test.shape:  (5, 987)


# Cooperative Coevolution

In [5]:
class individual:
    def __init__(self, lst):
        self.chromosome = lst
        self.fitness = 0
        self.involved_times = 0

In [6]:
# len(parentssss) == len(offspringsss)
def breed(parentssss):
    
    offspringsss = []
    chromosome_length = len(parentssss[0].chromosome)
    
    for i in range(int(len(parentssss) / 2)):
        parent1 = random.choice(parentssss)
        parent2 = random.choice(parentssss)

        # uniform crossover
        which_parent = np.random.uniform(size = chromosome_length)

        offspring1 = np.zeros(chromosome_length, dtype=int)
        offspring2 = np.zeros(chromosome_length, dtype=int)
        
        for k in range(chromosome_length):
            if which_parent[k] > 0.5:
                offspring1[k] = parent1.chromosome[k]
                offspring2[k] = parent2.chromosome[k]
            else:
                offspring1[k] = parent2.chromosome[k]
                offspring2[k] = parent1.chromosome[k]
                
        offspringsss.append(individual(list(offspring1)))
        offspringsss.append(individual(list(offspring2)))
    
    
    return offspringsss

In [7]:
def calculate_individual_fitness(chromosome):
    
    acc_sum = 0
    
    for d_num in range(5):
        clf = LogisticRegression(max_iter=400, tol=0.1, solver='saga').fit(x_train[:, chromosome], y_train[d_num])
        acc_sum += clf.score(x_val[:, chromosome], y_val[d_num])
    
    return acc_sum

In [8]:
# k: desired minimum number of tests per individual

def Access_Joint_fitness(five_populations):
    
    
    k_fold = 5
    
#     for population in five_populations:
    for pop_num in range(5):

        
        for each_individual in five_populations[pop_num]:
        
            
            # m <-- number of tests that this individual has been involved so far
            m = each_individual.involved_times
            
            # for w in (m+1) to k, do:
            for w in range(m, k_fold):
                
                s = [None]*10
                
                # for t in 1 to 5, do:
                for t in range(5):
                
                
                    # if t is theeee population:
                    if t == pop_num:
                    
                        s[2*t] = each_individual.chromosome[0]
                        s[2*t + 1] = each_individual.chromosome[1]
                        
                        each_individual.involved_times += 1
                        
                    else:
                        # s[t] = individaul chosen randomly from population_t
                        tmp = np.random.choice(five_populations[t])
                        
                        s[2*t] = tmp.chromosome[0]
                        s[2*t + 1] = tmp.chromosome[1]
                        
#                         tmp.involved_times += 1
                        
                # Test(s): Test the complete solution
                each_individual.fitness += calculate_individual_fitness(s)
                

In [9]:
generation = 1001
pop_size = 100
ell = 10
ell_each_pop = 2

current_best_chromosome_fitness_lst = []

# Build 5 initial Populations
G = [[], [], [], [], []]

for i in range(pop_size):
    G[0].append(individual(np.random.choice(list(range(0, 20)), ell_each_pop, replace=False)))
    G[1].append(individual(np.random.choice(list(range(20, 40)), ell_each_pop, replace=False)))
    G[2].append(individual(np.random.choice(list(range(40, 55)), ell_each_pop, replace=False)))
    G[3].append(individual(np.random.choice(list(range(55, 75)), ell_each_pop, replace=False)))
    G[4].append(individual(np.random.choice(list(range(75, 95)), ell_each_pop, replace=False)))


# Best <- Empty set
best = individual([])

# Repeat
for iteration in range(generation):

    # Breed
    G[0] = G[0] + breed(G[0])
    G[1] = G[1] + breed(G[1])
    G[2] = G[2] + breed(G[2])
    G[3] = G[3] + breed(G[3])
    G[4] = G[4] + breed(G[4])
    
    
    
    # Access_Joint_fitness
    Access_Joint_fitness(G)

    
    
    # selection
    G[0].sort(key=lambda x: x.fitness, reverse=True)
    G[1].sort(key=lambda x: x.fitness, reverse=True)
    G[2].sort(key=lambda x: x.fitness, reverse=True)
    G[3].sort(key=lambda x: x.fitness, reverse=True)
    G[4].sort(key=lambda x: x.fitness, reverse=True)
    
    G[0] = G[0][:len(G[0])//2]
    G[1] = G[1][:len(G[1])//2]
    G[2] = G[2][:len(G[2])//2]
    G[3] = G[3][:len(G[3])//2]
    G[4] = G[4][:len(G[4])//2]
    
    # choose the best chromosome in each population to form the best biomarkers
    tmp = list(G[0][0].chromosome) + list(G[1][0].chromosome) + list(G[2][0].chromosome) + list(G[3][0].chromosome) + list(G[4][0].chromosome)
    print(tmp)
    tmp_fitness = calculate_individual_fitness(tmp)
    print(f'{iteration}th tmp_fitness: {tmp_fitness}')

    current_best_chromosome_fitness_lst.append(tmp_fitness)
    
    if tmp_fitness > best.fitness:
        best.chromosome = tmp.copy()
        best.fitness = tmp_fitness
    
# stop until max iteration


KeyboardInterrupt: 

# print best chromosome, best acc, and predict test2.xls

In [13]:
# with open('./cooperative_coevolution_tmp_fitness.txt', 'w') as f:
#     for i in current_best_chromosome_fitness_lst:
#         f.write("%s\n" % i)

#     f.write("best chromosome: %s\n" % best.chromosome)
#     f.write("best fitness: %s\n" % best.fitness)

In [10]:
def two_ratio(num):
    i = 1
    k = 94
    num = num - 95
    while num > k:
        
        num = num - k
        i += 1
        k -= 1
        
    return i, num + i

In [13]:
best_biomarkers = open('./result/cooperative_coevolution_tmp_fitness.txt', 'r').readlines()[-2][18:-2].split(',')
best_biomarkers = list(map(int, best_biomarkers))
print(best_biomarkers)

[12, 15, 26, 26, 41, 49, 73, 65, 75, 77]


In [14]:
def predict(chromosome, d_num):
    clf = LogisticRegression(max_iter=400, tol=0.1, solver='saga').fit(x_train[:, chromosome], y_train[d_num])
    return clf.predict(x_test[:, chromosome])

In [15]:
y_test = []
for i in range(5):
    Test2[f'C0{i+1}'] = predict(best_biomarkers, i)
    
Test2.to_excel("./Midterm_Data/Test2_coop_result.xlsx", index=False) 